# Hochschule Bonn-Rhein-Sieg

# Mathematics for Robotics and Control

# Assignment 09 - Recursive Bayesian Filtering

Author(s)/team members:
* 
*
*

YOUR ANSWER HERE

In [2]:
import numpy as np
import IPython
import matplotlib.pyplot as plt

### Exercise 1: Is it hot or cold outside? [30 points]

Your first exercise for today is to once again solve the problem we looked at during the lab class (with very minor modifications). The problem setup is as follows: Let's suppose that we have a smart home with a temperature sensor in it, such that we want to predict whether it's hot or cold outside. Our temperature sensor measures the temperature unreliably: if the temperature is high, the sensor will report that it is high in $95\%$ of the cases; similarly, if the temperature is low, the sensor will report a high temperature in $10\%$ of the cases. Based on some old statistics, we know that if the temperature is high on day $t$, it will be high on day $t+1$ in $70\%$ of the cases; similarly, if the temperature is low on day $t$, there is a $20\%$ chance that it will be high on day $t+1$.

a) [2 points] Model the problem mathematically and, using your notation, write down the equation for the Bayes filter that will allow us to estimate the outside temperature at time $t+1$ based on the estimates up to time $t$ and measurements up to time $t+1$.

Let us take,
\begin{align*}
    T &\implies Temperature \\
    S &\implies SensorReport
\end{align*}
From the problem description,we know that,
\begin{align*}
    P(S = high | T = high) &= 0.95 &\ P(S = low | T = high) &= 0.05 \\
    P(S = high | T = low) &= 0.1 & P(S = low | T = low) &= 0.9 \\
    P(T_{t+1} = high | T_t = high) &= 0.7 & P(T_{t+1} = low | T_t = high) &= 0.3 \\
    P(T_{t+1} = high | T_t = low) &= 0.2 & P(T_{t+1} = low | T_t = low) &= 0.8
\end{align*}

Equation for the Bayes filter 
\begin{equation*}
    P(T_{t+1} = high|S_{t+1} = high) = \alpha P(S_{t+1} = high|T_{t+1} = high) \sum_{t}{P(T_{t+1} = high|t) P(t)}
\end{equation*}



b) [5 points] Suppose that the sensor reports a low temperature on day $t = 1$. Calculate the probability (the actual probability, not just up to a normalising constant) that the outside temperature on day $t = 1$ is high. Assume that we are fully ignorant about the outside temperature on day $t = 0$ (i.e. assume that every possible outside temperature is equally likely).

To calculate the probability that the outside temperature on day $t = 1$ is high:
\begin{align*}
   P(T_{1} = high|S_{1} = low) &= \alpha P(S_{1} = low|T_{1} = high) \sum_{t_0}{P(T_{1} = high|t_0) P(t_0)} \\
    &= \alpha \cdot 0.05 \cdot (0.7 \cdot 0.5 +0.2 \cdot 0.5) \\
    &= \alpha \cdot (0.0225) \\
    P(T_1 = low|S_1 = low) &= \alpha P(S_1 = low|T_1 = low) \sum_{t_0}{P(T_1 = low|t_0) P(t_0)} \\
    &= \alpha \cdot 0.9 \cdot (0.3 \cdot 0.5 +0.8 \cdot 0.5) \\
    &= \alpha \cdot (0.495)
\end{align*}
By normalizing, 
\begin{align*}
    P(T_1 = high|S_1 = low) &= \alpha \cdot (0.0225) =0.04348 \\
    P(T_1 = low|S_1 = low) &= \alpha \cdot (0.495) = 0.9565
\end{align*}

c) [3 points] If the sensor reports a high temperature on day $t = 2$, what is the probability that the outside temperature is indeed high?

at day $ t =1$
\begin{equation*}
    P(T_{2} = high|S_{2} = high) = \alpha P(S_{2} = high|T_{2} = high) \sum_{t_1}{P(T_{2} = high|t_1) P(t_1)}
\end{equation*}

to calculate the probability that the outside temperature on day $t = 2$ is high:
\begin{align*}
    P(T_{2} = high|S_{2} = high) &= \alpha P(S_{2} = high|T_{2} = high) \sum_{t_1}{P(T_{2} = high|t_1) P(t_1)} \\
                                &= \alpha \cdot 0.95 \cdot (0.7 \cdot 0.04348 + 0.2 \cdot 0.9565) \\
                                &= \alpha \cdot (0.2106) \\
    P(T_{2} = low|S_{2} = high) &= \alpha P(S_{2} = high|T_{2} = low) \sum_{t_1}{P(T_{2} = high|t_1) P(t_1)}\\
                                &= \alpha \cdot 0.1 \cdot (0.3 \cdot 0.04348 +0.8 \cdot 0.9565) \\
                                 &= \alpha \cdot (0.07782)
\end{align*}
By normalizing, 
\begin{align*}
    P(T_2 = high|S_2 = high) &= \alpha \cdot (0.2106) = 0.7302 \\
    P(T_2 = low|S_2 = high) &= \alpha \cdot (0.07782) = 0.2697
\end{align*}

d) [20 points] In the cell below, write a function $measurement\_update$ that can be used for updating the belief about the outside temperature given a new measurement. Use this implementation to find the belief about the outside temperature at time $t = 10$ given the sequence of measurements

low, high, low, high, high, high, high, low, high, low

from $t = 1$ up to $t = 10$. Once again, assume that we are fully ignorant about the outside temperature on day $t = 0$. Note that the first two measurements are the same as in problems b) and c), so you can use the estimates at times $t=1$ and $t=2$ to cross-check your previous results.

In [38]:
# YOUR CODE HERE
events = ["high","low"]
measurements_seq = ['low', 'high', 'low', 'high', 'high', 'high', 'high', 'low', 'high', 'low']
P_S_T = {"high":{"high":0.95,"low":0.1}, "low":{"high":(1-0.95),"low":(1-0.1)}}
P_Tcurr_Tprev = {"high":{"high":0.7,"low":0.2}, "low":{"high":(1-0.7),"low":(1-0.2)}}
P_t = [{"high": 0.5, "low": 0.5}]

def measurement_update(S,t):
    probability = dict()
    for Tcurr in events:
        prior = 0
        for Tprev in events:
            # Prior - Control update
            prior += P_Tcurr_Tprev[Tcurr][Tprev]*P_t[t-1][Tprev]
        # sensor model* prior
        probability[Tcurr] = P_S_T[S][Tcurr]*prior
    return get_normalize(probability)

def get_normalize(P):
    '''To normalize the probabilities'''
    normaliser = 0
    for val in P.values(): normaliser += val
    for event in P : P[event] = P[event]/normaliser
    return P

def get_belief(curr_belief):
    '''compare the probalities and returns belief '''
    probabilities = list(curr_belief.values()) 
    events = list(curr_belief.keys()) 
    # event with maximum probability
    belief = events[probabilities.index(max(probabilities))]
    return belief
    
for measurement,t in zip(measurements_seq,range(1,len(measurements_seq)+1)):
    P_t.append(measurement_update(measurement,t))
    print("At the day ",t," measurement from the sensor is ",measurement," and the outside temperature is believe to be ",get_belief(P_t[t]))

# for i in P_t : print(i)

At the day  1  measurement from the sensor is  low  and the outside temperature is believe to be  low
At the day  2  measurement from the sensor is  high  and the outside temperature is believe to be  high
At the day  3  measurement from the sensor is  low  and the outside temperature is believe to be  low
At the day  4  measurement from the sensor is  high  and the outside temperature is believe to be  high
At the day  5  measurement from the sensor is  high  and the outside temperature is believe to be  high
At the day  6  measurement from the sensor is  high  and the outside temperature is believe to be  high
At the day  7  measurement from the sensor is  high  and the outside temperature is believe to be  high
At the day  8  measurement from the sensor is  low  and the outside temperature is believe to be  low
At the day  9  measurement from the sensor is  high  and the outside temperature is believe to be  high
At the day  10  measurement from the sensor is  low  and the outside t

### Exercise 2: Lucy, tell me a joke [50 points]

Our robot Lucy has a trained model that it can use for recognising human emotions; the actual model is able to recognise seven emotions in total, but for the sake of this problem, let us assume that it can only recognise three - sad, angry, and happy - which we will denote by $R_S$, $R_A$, and $R_H$ respectively. Unfortunately, the model is not entirely reliable and often recognises incorrect emotions; in particular, let us assume that we are given the following observation probabilities:

\begin{align*}
    &P(R_{S,t}|E_{S,t}) = 0.8 \hspace{2.1cm} P(R_{A,t}|E_{S,t}) = 0.15 \hspace{2cm} P(R_{H,t}|E_{S,t}) = 0.05 \\
    &P(R_{S,t}|E_{A,t}) = 0.2 \hspace{2cm} P(R_{A,t}|E_{A,t}) = 0.7 \hspace{2.2cm} P(R_{H,t}|E_{A,t}) = 0.1 \\
    &P(R_{S,t}|E_{H,t}) = 0.02 \hspace{1.75cm} P(R_{A,t}|E_{H,t}) = 0.03 \hspace{1.95cm} P(R_{H,t}|E_{H,t}) = 0.95 \\
\end{align*}

where $E$ denotes the actual emotion and $t$ denotes time. Now, our robot wants to be a useful companion to its owner and can communicate with them, namely it can tell a joke, show a calming picture on its screen, and laugh lightly, events that we will denote by $C_J$, $C_P$, and $C_L$ respectively. For a particular owner (as in one of the previous assignments, let's assume it's Alex), the following transition probabilities are known:

\begin{align*}
    &P(E_{S,t}|C_{J,t}, E_{S,t-1}) = 0.1 \hspace{2.1cm} P(E_{A,t}|C_{J,t}, E_{S,t-1}) = 0.01 \hspace{2cm} P(E_{H,t}|C_{J,t}, E_{S,t-1}) = 0.89 \\
    &P(E_{S,t}|C_{J,t}, E_{A,t-1}) = 0.02 \hspace{1.82cm} P(E_{A,t}|C_{J,t}, E_{A,t-1}) = 0.1 \hspace{2.2cm} P(E_{H,t}|C_{J,t}, E_{A,t-1}) = 0.88 \\
    &P(E_{S,t}|C_{J,t}, E_{H,t-1}) = 0.01 \hspace{1.78cm} P(E_{A,t}|C_{J,t}, E_{H,t-1}) = 0 \hspace{2.5cm} P(E_{H,t}|C_{J,t}, E_{H,t-1}) = 0.99 \\~\\
    &P(E_{S,t}|C_{P,t}, E_{S,t-1}) = 0.45 \hspace{1.75cm} P(E_{A,t}|C_{P,t}, E_{S,t-1}) = 0 \hspace{2.6cm} P(E_{H,t}|C_{P,t}, E_{S,t-1}) = 0.55 \\
    &P(E_{S,t}|C_{P,t}, E_{A,t-1}) = 0.55 \hspace{1.73cm} P(E_{A,t}|C_{P,t}, E_{A,t-1}) = 0.1 \hspace{2.22cm} P(E_{H,t}|C_{P,t}, E_{A,t-1}) = 0.35 \\
    &P(E_{S,t}|C_{P,t}, E_{H,t-1}) = 0.01 \hspace{1.7cm} P(E_{A,t}|C_{P,t}, E_{H,t-1}) = 0.01 \hspace{1.92cm} P(E_{H,t}|C_{P,t}, E_{H,t-1}) = 0.98 \\~\\
    &P(E_{S,t}|C_{L,t}, E_{S,t-1}) = 0.2 \hspace{1.92cm} P(E_{A,t}|C_{L,t}, E_{S,t-1}) = 0.1 \hspace{2.41cm} P(E_{H,t}|C_{L,t}, E_{S,t-1}) = 0.7 \\
    &P(E_{S,t}|C_{L,t}, E_{A,t-1}) = 0.1 \hspace{1.92cm} P(E_{A,t}|C_{L,t}, E_{A,t-1}) = 0.4 \hspace{2.4cm} P(E_{H,t}|C_{L,t}, E_{A,t-1}) = 0.5 \\
    &P(E_{S,t}|C_{L,t}, E_{H,t-1}) = 0 \hspace{2.22cm} P(E_{A,t}|C_{L,t}, E_{H,t-1}) = 0 \hspace{2.65cm} P(E_{H,t}|C_{L,t}, E_{H,t-1}) = 1
\end{align*}

a) [10 points] Write down the equation for the Bayes filter that will allow us to estimate Alex's emotion at time $t+1$ based on the estimates up to time $t$, the measurements up to time $t+1$, and Lucy's communication actions up to time $t + 1$. Then, use the Bayes filter to manually calculate the probabilities of the different emotions at time $t = 1$ given that Lucy has told a joke and then seen Alex being happy. Assume that Alex is angry at time $t = 0$.
</div>

To calculate the probability that the outside temperature on day $t = 1$ is high:
\begin{align*}
   P(T_{1} = high|S_{1} = low) &= \alpha P(S_{1} = low|T_{1} = high) \sum_{t_0}{P(T_{1} = high|t_0) P(t_0)} \\
    &= \alpha \cdot 0.05 \cdot (0.7 \cdot 0.5 +0.2 \cdot 0.5) \\
    &= \alpha \cdot (0.0225) \\
    P(T_1 = low|S_1 = low) &= \alpha P(S_1 = low|T_1 = low) \sum_{t_0}{P(T_1 = low|t_0) P(t_0)} \\
    &= \alpha \cdot 0.9 \cdot (0.3 \cdot 0.5 +0.8 \cdot 0.5) \\
    &= \alpha \cdot (0.495)
\end{align*}
By normalizing, 
\begin{align*}
    P(T_1 = high|S_1 = low) &= \alpha \cdot (0.0225) =0.04348 \\
    P(T_1 = low|S_1 = low) &= \alpha \cdot (0.495) = 0.9565
\end{align*}

b) [30 points] In the cell below, implement two functions - $measurement\_update$ and $control\_update$ - which can be used for recursively updating the belief about Alex's emotion based on all previous estimates, measurements, and robot actions. Use your implementation to estimate the beliefs about Alex's emotions if we assume the following sequence of actions and emotion observations:

*tell a joke*, happy, *show a picture*, sad, *tell a joke*, angry, *laugh*, happy, *show a picture*, sad, *tell a joke*, happy, *show a picture*, happy

Once again, assume that Alex is angry at time $t = 0$. Note that the first action-measurement pair is the same as in a), so you can use the estimate to cross-check your previous result.

In [37]:
# YOUR CODE HERE

emotions = ['S','A','H']
communications = ['J','P','L']
sequence = ['J','H','P','S','J','A','L','H','P','S','J','H','P','H']

P_R_E = {'S':{'S': 0.8  ,'A':0.2 ,'H':0.02},
         'A':{'S': 0.15 ,'A':0.7 ,'H':0.03},
         'H':{'S': 0.05 ,'A':0.1 ,'H':0.95}}

P_E_C_E = {'S':{'J':{'S': 0.10 ,'A':0.02 ,'H':0.01} ,
                'P':{'S': 0.45 ,'A':0.55 ,'H':0.01} ,
                'L':{'S': 0.20 ,'A':0.10 ,'H':0.00}},
           'A':{'J':{'S': 0.01 ,'A':0.10 ,'H':0.00} ,
                'P':{'S': 0.00 ,'A':0.10 ,'H':0.01} ,
                'L':{'S': 0.10 ,'A':0.40 ,'H':0.00}},
           'H':{'J':{'S': 0.89 ,'A':0.88 ,'H':0.99} ,
                'P':{'S': 0.55 ,'A':0.35 ,'H':0.98} ,
                'L':{'S': 0.70 ,'A':0.50 ,'H':1.00}}}

P_t = [{'S': 0.0, 'A': 1.0, 'H': 0.0}]

def measurement_update(curr_belief,Reaction):
    probability = dict()
    for E in emotions:
        probability[E] = P_R_E[Reaction][E]*curr_belief[E]
    return get_normalize(probability)

def control_update(Action,t):
    updated_belief = dict()
    for E in emotions:
        prior = 0
        for e in emotions:
            # Prior
            prior += P_E_C_E[E][Action][e]*P_t[t-1][e]
        updated_belief[E] = prior
    return get_normalize(updated_belief)

t = 1
for element in sequence:
    if element in communications:
        curr_belief = control_update(element,t)
    elif element in emotions:
        P_t.append(measurement_update(curr_belief,element))
        t += 1
        
for t,i in enumerate(P_t): print(" At t =",t,"Belief =",get_belief(i),", Sad =",i['S'],", Angry =",i['A'],", Happy =",i['H'])  


 At t = 0 Belief = A , Sad = 0.0 , Angry = 1.0 , Happy = 0.0
 At t = 1 Belief = H , Sad = 0.001180637544273908 , Angry = 0.011806375442739082 , Happy = 0.987012987012987
 At t = 2 Belief = H , Sad = 0.38433389510719584 , Angry = 0.06284705203010749 , Happy = 0.5528190528626965
 At t = 3 Belief = H , Sad = 0.16068360219035507 , Angry = 0.16795292507990123 , Happy = 0.6713634727297437
 At t = 4 Belief = H , Sad = 0.00292936221258385 , Angry = 0.009967627058443108 , Happy = 0.987103010728973
 At t = 5 Belief = H , Sad = 0.38149826242112367 , Angry = 0.06217286232702101 , Happy = 0.5563288752518554
 At t = 6 Belief = H , Sad = 0.002494783054087943 , Angry = 0.0011134449576812087 , Happy = 0.9963917719882308
 At t = 7 Belief = H , Sad = 0.0006283638075015416 , Angry = 0.0010823060372089047 , Happy = 0.9982893301552895


b) [10 points] If Lucy keeps acting without observing Alex's emotions, what do you think will happen to the emotion estimates in the long run? Note that acting without observations is the same as only performing a control update, i.e. equivalent to only performing a prediction step in the Bayes filter, but without a measurement correction.

Verify your hypothesis in the cell below by generating random sequences of actions and updating the emotion estimates only based on the control updates.

In [31]:
# YOUR CODE HERE
P_t = [{'S': 0.0, 'A': 1.0, 'H': 0.0}]

# Random sequence 
sequence = [np.random.choice(communications) for i in range(10)]

t = 1
for element in sequence:
    if element in communications:
        curr_belief = control_update(element,t)
        P_t.append(curr_belief)
        t += 1
        
print("Randomly generated sequence : ",sequence)
for t,i in enumerate(P_t): print(" At t =",t,"Belief =",get_belief(i),", Sad =",i['S'],", Angry =",i['A'],", Happy =",i['H']) 

Randomly generated sequence :  ['L', 'P', 'J', 'L', 'P', 'J', 'L', 'P', 'L', 'J']
 At t = 0 Belief = A , Sad = 0.0 , Angry = 1.0 , Happy = 0.0
 At t = 1 Belief = H , Sad = 0.1 , Angry = 0.4 , Happy = 0.5
 At t = 2 Belief = H , Sad = 0.27 , Angry = 0.045000000000000005 , Happy = 0.685
 At t = 3 Belief = H , Sad = 0.03475 , Angry = 0.007200000000000001 , Happy = 0.9580500000000001
 At t = 4 Belief = H , Sad = 0.0076700000000000015 , Angry = 0.006355000000000001 , Happy = 0.985975
 At t = 5 Belief = H , Sad = 0.016806500000000002 , Angry = 0.010495250000000001 , Happy = 0.9726982500000001
 At t = 6 Belief = H , Sad = 0.011617537500000002 , Angry = 0.0012175900000000002 , Happy = 0.9871648725000001
 At t = 7 Belief = H , Sad = 0.0024452665000000004 , Angry = 0.0016487897500000004 , Happy = 0.9959059437500001
 At t = 8 Belief = H , Sad = 0.011966263725000002 , Angry = 0.0101239384125 , Happy = 0.9779097978625
 At t = 9 Belief = H , Sad = 0.003405646586250001 , Angry = 0.005246201737500001 ,

YOUR ANSWER HERE